* Initialize the libraries and variables

In [ ]:
import { DOMParser } from "https://deno.land/x/deno_dom@v0.1.38/deno-dom-wasm.ts";
import { logError } from "./logError.js";
import { reloadEnv } from "./reloadEnv.js"
await reloadEnv();

const token = Deno.env.get("AEM_TOKEN");
const aemOrigin = 'https://author-p22655-e59433.adobeaemcloud.com';
const destination = '/content/dam/mas/acom/en_US/';

* Test if the token is working

In [ ]:
const url = `${aemOrigin}/bin/querybuilder.json?path=/content/dam/mas&path.flat=true&type=sling:Folder&p.limit=-1`;

const response = await fetch(url, {
  headers: {
    "Authorization": `Bearer ${token}`
  }
});

if (!response.ok) {
  logError(response);
}

console.log(response.status);

* Read card id to be copied from a file

In [ ]:
const idListFilename = './copy_idlist.txt';
const idList = (await Deno.readTextFile(idListFilename)).split('\n');
console.log(`Id count: ${idList.length}`);
console.log(idList.join('\n'));

* Fetch all cards to be copied

In [ ]:
const cardsToBeCopied = [];

for (const item of idList) {
  const cardId = item;
  const url = `${aemOrigin}/adobe/sites/cf/fragments/${cardId}`;


  // Get the card
  const response = await fetch(url, {
    headers: {
      "x-api-key": "mas-studio",
      "Authorization": `Bearer ${token}`,
      pragma: 'no-cache',
      'cache-control': 'no-cache',
      'x-aem-affinity-type': 'api',
    }
  });

  if (!response.ok) {
    logError(response);
  } else {
    const data = await response.json();
    cardsToBeCopied.push(data);
  }
}

console.log(`Total cards: ${cardsToBeCopied.length}`);
console.log(JSON.stringify(cardsToBeCopied[0], null, 2));


* Save to the destination

In [ ]:
const cardsDone = [];
const cardsFailed = [];

for (const item of cardsToBeCopied) {
  const cardId = item.id;
  const url = `${aemOrigin}/adobe/sites/cf/fragments/${cardId}`;

  console.log(`Getting card ${cardId}`);

  // Get the card
  const response = await fetch(url, {
    headers: {
      "x-api-key": "mas-studio",
      "Authorization": `Bearer ${token}`,
      pragma: 'no-cache',
      'cache-control': 'no-cache',
      'x-aem-affinity-type': 'api',
    }
  });

  if (!response.ok) {
    logError(response);
    cardsFailed.push(item);
    continue;
  }
  const data = await response.json();

  // Save to the destination
  const parentPath = destination;
  const name = data.path.replace(/^.*[\\\/]/, '');

  console.log(`Saving card ${name} to ${parentPath}`);

  const title = data.title;
  const modelId = data.model.id;
  const description = data.description;
  const fields = data.fields;
  
  const urlSave = `${aemOrigin}/adobe/sites/cf/fragments`;

  const responseSave = await fetch(urlSave, {
    method: 'POST',
    headers: {
      'Content-Type': 'application/json',
      "x-api-key": "mas-studio",
      "Authorization": `Bearer ${token}`,
      'x-aem-affinity-type': 'api',
    },
    body: JSON.stringify({
      title,
      name,
      modelId,
      parentPath,
      description,
      fields,
    }),
  })

  if (!responseSave.ok) {
    logError(responseSave);
    cardsFailed.push(item);
    console.log(`Failed to save card ${name} to ${parentPath}`);
  } else {
    console.log(`Saving card ${name} to ${parentPath}`);
    const data = await responseSave.json();
    cardsDone.push(data);
  } 
}

console.log(`Total cards copied: ${cardsDone.length}`);
console.log(`Total cards failed: ${cardsFailed.length}`);